In [89]:
import pandas as pd
import numpy as np
import nltk
import ssl
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to /Users/ryanluu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [90]:
! pip3 install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


## Read Data

In [91]:
data=pd.read_table('amazon_reviews_us_Jewelry_v1_00.tsv',usecols = ['star_rating','review_body'], low_memory=False)

## Keep Reviews and Ratings

In [92]:
# Done already in prior line using "usecols = ['star_rating, 'review_body']"

 ## We select 20000 reviews randomly from each rating class.



In [93]:
sampled_amazon_df = data.sample(n=20000)

# Data Cleaning



# Pre-processing

In [88]:

#sampled_amazon_df.dropna(subset=['review_body'],axis=0,inplace=True)
#sampled_amazon_df.isna().sum()/len(sampled_amazon_df)*100

sampled_amazon_df.dtypes

print(sampled_amazon_df)

        star_rating                                        review_body
807114            4  This is a great ring, however, the photo is so...
1162168           3  It looks better in pictures than when you get ...
544887            5                                           Adorable
553616            4                        Beautiful piece of jewelry.
59993             5           So pretty, just what I was looking for !
...             ...                                                ...
772279            5  Mom loved this necklace as a mother's day pres...
1065714           4  They are as shown in the picture. They have a ...
853702            3  They really smell weird but it'll probably wea...
1193469           4  The little girl I bought it for loved them! Bu...
133307            5                                        looks great

[19998 rows x 2 columns]


## remove the stop words 

In [ ]:
from nltk.corpus import stopwords
 

## perform lemmatization  

In [ ]:
from nltk.stem import WordNetLemmatizer


# TF-IDF Feature Extraction

# Perceptron

# SVM

# Logistic Regression

# Naive Bayes